<a href="https://colab.research.google.com/github/WafleraVertical/Clasificaci-n-Musical-por-Genero/blob/main/Copia_de_CNN_con_Espectrogramas_Mel_(Han_et_al_%2C_2016).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import librosa
import librosa.display
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import VGG16


In [ ]:
def extract_spectrogram(file_path, n_mels=128, fmax=8000):
    y, sr = librosa.load(file_path, sr=None)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, fmax=fmax)
    log_S = librosa.power_to_db(S, ref=np.max)
    return log_S

In [ ]:
def load_gtzan_data(gtzan_path, genres, n_mels=128):
    X = []
    y = []
    for genre in genres:
        genre_dir = os.path.join(gtzan_path, genre)
        for file_name in os.listdir(genre_dir):
            file_path = os.path.join(genre_dir, file_name)
            spectrogram = extract_spectrogram(file_path, n_mels=n_mels)
            if spectrogram.shape[1] > 128:
                spectrogram = spectrogram[:, :128]
            elif spectrogram.shape[1] < 128:
                spectrogram = np.pad(spectrogram, ((0, 0), (0, 128 - spectrogram.shape[1])), mode='constant')

            X.append(spectrogram)
            y.append(genres.index(genre))
    return np.array(X), np.array(y)

In [ ]:
GTZAN_PATH = '/content/drive/MyDrive/Tesis/GZTAN/genres_original'
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

In [ ]:
X, y = load_gtzan_data(GTZAN_PATH, genres)

In [ ]:
X = X[..., np.newaxis]
X = X / np.max(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')


In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test),
          callbacks=[early_stopping, model_checkpoint])



Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 884ms/step - accuracy: 0.1039 - loss: 35954564.0000 - val_accuracy: 0.0700 - val_loss: 2.2915
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 22s 883ms/step - accuracy: 0.1136 - loss: 4.4244 - val_accuracy: 0.0650 - val_loss: 2.9633
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 857ms/step - accuracy: 0.0992 - loss: 2.9270 - val_accuracy: 0.0650 - val_loss: 3.5427
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 857ms/step - accuracy: 0.1184 - loss: 2.3091 - val_accuracy: 0.0650 - val_loss: 3.5935
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 926ms/step - accuracy: 0.1039 - loss: 2.2979 - val_accuracy: 0.0650 - val_loss: 3.5982
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 39s 846ms/step - accuracy: 0.1231 - loss: 2.2984 - val_accuracy: 0.0650 - val_loss: 3.5991


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Precisión en test: {test_acc:.2f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 171ms/step - accuracy: 0.0724 - loss: 2.2966
Precisión en test: 0.07
